In [37]:
import random
from sympy import isprime, mod_inverse

def generate_large_prime(bits):
    """Generate a large prime number."""
    while True:
        prime_candidate = random.getrandbits(bits)
        if isprime(prime_candidate):
            return prime_candidate

def primitive_root_generator(p):
    """Generate a primitive root modulo p."""
    while True:
        g = random.randint(2, p - 2)
        if pow(g, (p - 1) // 2, p) != 1:
            return g

def elgamal_key_generation(bits=128):
    """ElGamal Key Generation."""
    # Step 1: Select a large prime number p
    p = generate_large_prime(bits)

    # Step 2: Choose a secret integer d (1 ≤ d ≤ p - 2)
    d = random.randint(1, p - 2)

    # Step 3: Select e1, a primitive root modulo p
    e1 = primitive_root_generator(p)

    # Step 4: Compute e2 = e1^d mod p
    e2 = pow(e1, d, p)

    # Public Key: (e1, e2, p)
    public_key = (e1, e2, p)

    # Private Key: d
    private_key = d

    return public_key, private_key

# Example usage
public_key, private_key = elgamal_key_generation()

print({
    "e1": public_key[0],
    "e2": public_key[1],
    "p": public_key[2],
    "d": private_key
})
print("Public Key:", public_key)
print("Private Key:", private_key)

{'e1': 55076296596830904199741903398955120404, 'e2': 106373456543642511198487400642046542696, 'p': 288634617432859657795895109735198534577, 'd': 72546442531488452052137253222119926088}
Public Key: (55076296596830904199741903398955120404, 106373456543642511198487400642046542696, 288634617432859657795895109735198534577)
Private Key: 72546442531488452052137253222119926088


In [40]:
# To check if plaintext is non-integer type
IS_STRING = False

def elgamal_encryption(public_key, plaintext):
    """ElGamal Encryption."""
    global IS_STRING
    e1, e2, p = public_key

    # Step 1: Select a random integer r in [1, p - 2]
    r = random.randint(1, p - 2)

    # Step 2: Compute C1 = e1^r mod p
    C1 = pow(e1, r, p)

    # Optional step to convert string to integer
    if not plaintext.isdigit():
        IS_STRING = True
        plaintext = int.from_bytes(plaintext.encode(), 'big')

    # Step 3: Compute C2 = (P * e2^r) mod p
    C2 = (plaintext * pow(e2, r, p)) % p

    # Return ciphertext (C1, C2)
    return (C1, C2)

# Usage
plaintext = input("Enter the plaintext to encrypt: ")
ciphertext = elgamal_encryption(public_key, plaintext)
print({
    "C1": ciphertext[0],
    "C2": ciphertext[1],
    "IS_STRING": IS_STRING
})
print("Ciphertext:", ciphertext)

Enter the plaintext to encrypt: Hello World!
{'C1': 166887197058045896588892203676713310171, 'C2': 35685365510596946089659527090488838350, 'IS_STRING': True}
Ciphertext: (166887197058045896588892203676713310171, 35685365510596946089659527090488838350)


In [42]:
def elgamal_decryption(private_key, public_key, ciphertext):
    """ElGamal Decryption."""
    d = private_key
    e1, e2, p = public_key
    C1, C2 = ciphertext

    # Step 1: Compute C1^d mod p
    C1_d = pow(C1, d, p)

    # Step 2: Compute the inverse of C1^d mod p
    C1_d_inverse = mod_inverse(C1_d, p)

    # Step 3: Calculate plaintext P = (C2 * (C1^d)^-1) mod p
    plaintext = (C2 * C1_d_inverse) % p

    return plaintext

# Usage
decrypted_plaintext = elgamal_decryption(private_key, public_key, ciphertext)
if IS_STRING:
    decrypted_plaintext = decrypted_plaintext.to_bytes((decrypted_plaintext.bit_length() + 7) // 8, 'big').decode()

print(f"Decrypted plaintext: {decrypted_plaintext}")

Decrypted plaintext: Hello World!
